In [1]:
import psycopg2 as ps
import pandas as pd

In [2]:
df = pd.read_csv('youtubeData.csv', index_col = 0)
df.head()

,video_id,video_title,video_date,view_count,like_count,comment_count
0,Hwybp38GnZw,I Built Willy Wonka&#39;s Chocolate Factory!,2022-06-04 20:00:02+00:00,72184676,3955922,167574
1,hD1YtmKXNb4,"$10,000 Every Day You Survive Prison",2022-04-09 20:00:00+00:00,71950015,3163669,155042
2,LnlKwzc_TNA,"Would You Swim With Sharks For $100,000?",2022-03-19 20:02:32+00:00,65759316,2554696,101023
3,3jS_yEK8qVI,World’s Most Dangerous Escape Room!,2022-01-29 21:00:01+00:00,105450397,3613436,143352
4,00NgUctWoLQ,"Extreme $1,000,000 Hide And Seek",2021-12-18 21:00:00+00:00,67451827,3093434,136564


In [3]:
#connect to db
def connect_to_db(host_name, dbname, username, password, port):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
    
    return conn

#create table
def create_table(curr):
    
    create_table_command = (''' CREATE TABLE IF NOT EXISTS videos (
                            video_id VARCHAR(255) PRIMARY KEY,
                            video_title TEXT NOT NULL,
                            video_date DATE NOT NULL DEFAULT CURRENT_DATE,
                            view_count INTEGER NOT NULL,
                            like_count INTEGER NOT NULL,
                            comment_count INTEGER NOT NULL
                            )''')
    
    curr.execute(create_table_command)


#insert videos that arent on the DB
def insert_into_table(curr, video_id, video_title, video_date, view_count, like_count, comment_count):    
    insert_into_videos = ('''INSERT INTO videos (video_id, video_title, video_date, view_count, like_count, comment_count)
        VALUES(%s,%s,%s,%s,%s,%s)''')
    row_to_insert = (video_id, video_title, video_date, view_count, like_count, comment_count)

    curr.execute(insert_into_videos, row_to_insert)
    

#update row if video exists
def update_row(curr, video_id, video_title, view_count, like_count, dislike_count, comment_count):
    query = ('''UPDATE videos
                SET video_title = %s,
                    view_count = %s,
                    like_count = %s,
                    comment_count = %s
                WHERE video_id = %s;''')

    vars_to_update = (video_title, view_count, like_count, comment_count, video_id)
    curr.execute(query, vars_to_update)

#check if video exists
def check_if_video_exists(curr, video_id):
    query = ('''SELECT video_id FROM videos WHERE video_id = %s''')
    curr.execute(query, (video_id,))
    
    return curr.fetchone() is not None


def truncate_table(curr):
    truncate_table = ('''TRUNCATE TABLE videos''')
    
    curr.execute(truncate_table)
    
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr, row['video_id'], row['video_title'], row['video_date'], row['view_count'], row['like_count'], row['comment_count'])
        
def update_db(curr, df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'video_date', 'view_count', 'like_count', 'comment_count'])
    
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']):
            update_row(curr, row['video_id'], row['video_title'],row['view_count'],row['like_count'],row['dislike_count'],row['comment_count'])
        else:
            tmp_df = tmp_df.append(row)
            
    return tmp_df


In [4]:
#updating the database
#handling data for scalability

#check to see if video exists

#perform update on existing videos
#perform an insert on new videos

In [13]:
host_name= 'database-1.clwa0hdqmgzj.us-east-1.rds.amazonaws.com'
dbname= 'aws_rds_db'
port= '5432'
username= 'postgres'
password= 'senha123'

#Establishing connection to the db
conn = connect_to_db(host_name, dbname, username, password, port)
curr = conn.cursor()

Connected!


In [6]:
create_table(curr)

In [7]:
new_vid_df = update_db(curr,df)
conn.commit()

C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\A

C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\A

C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\A

C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\A

C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\A

C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\A

C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\A

C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\AppData\Local\Temp\ipykernel_11692\1167786295.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_df = tmp_df.append(row)
C:\Users\Lucas D. Cypriano\A

In [8]:
append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [9]:
curr.execute('''SELECT * FROM videos''')
print(curr.fetchall())

[('Hwybp38GnZw', 'I Built Willy Wonka&#39;s Chocolate Factory!', datetime.date(2022, 6, 4), 72184676, 3955922, 167574), ('hD1YtmKXNb4', '$10,000 Every Day You Survive Prison', datetime.date(2022, 4, 9), 71950015, 3163669, 155042), ('LnlKwzc_TNA', 'Would You Swim With Sharks For $100,000?', datetime.date(2022, 3, 19), 65759316, 2554696, 101023), ('3jS_yEK8qVI', 'World’s Most Dangerous Escape Room!', datetime.date(2022, 1, 29), 105450397, 3613436, 143352), ('00NgUctWoLQ', 'Extreme $1,000,000 Hide And Seek', datetime.date(2021, 12, 18), 67451827, 3093434, 136564), ('x9TQ6culXIA', '$1,000,000 Influencer Tournament!', datetime.date(2021, 12, 16), 53922528, 2741547, 92027), ('QbJJwaVdgIs', 'I Got Hunted By The Military', datetime.date(2021, 12, 10), 73805261, 2906973, 96680), ('0e3GPea1Tyg', '$456,000 Squid Game In Real Life!', datetime.date(2021, 11, 24), 262715148, 13889547, 609509), ('cV2gBU6hKfY', 'I Cleaned The World’s Dirtiest Beach #TeamSeas', datetime.date(2021, 10, 29), 64837254, 45

In [10]:
curr.close()

In [11]:
curr = conn.cursor()

In [14]:
curr.execute('''SELECT * FROM videos''')
print(curr.fetchall())

[('Hwybp38GnZw', 'I Built Willy Wonka&#39;s Chocolate Factory!', datetime.date(2022, 6, 4), 72184676, 3955922, 167574), ('hD1YtmKXNb4', '$10,000 Every Day You Survive Prison', datetime.date(2022, 4, 9), 71950015, 3163669, 155042), ('LnlKwzc_TNA', 'Would You Swim With Sharks For $100,000?', datetime.date(2022, 3, 19), 65759316, 2554696, 101023), ('3jS_yEK8qVI', 'World’s Most Dangerous Escape Room!', datetime.date(2022, 1, 29), 105450397, 3613436, 143352), ('00NgUctWoLQ', 'Extreme $1,000,000 Hide And Seek', datetime.date(2021, 12, 18), 67451827, 3093434, 136564), ('x9TQ6culXIA', '$1,000,000 Influencer Tournament!', datetime.date(2021, 12, 16), 53922528, 2741547, 92027), ('QbJJwaVdgIs', 'I Got Hunted By The Military', datetime.date(2021, 12, 10), 73805261, 2906973, 96680), ('0e3GPea1Tyg', '$456,000 Squid Game In Real Life!', datetime.date(2021, 11, 24), 262715148, 13889547, 609509), ('cV2gBU6hKfY', 'I Cleaned The World’s Dirtiest Beach #TeamSeas', datetime.date(2021, 10, 29), 64837254, 45